In [1]:
addprocs(4);

In [2]:
using JLD, PyPlot, MAT
@everywhere using module_DDM_v4
using analysis_functions
using Pandas, helpers
@everywhere using opt_diff_model, Optim, LineSearches

In [3]:
pth=ENV["HOME"]*"/Dropbox/spike-data_latent-accum";

In [111]:
#tail="18323739/8190069"; model_type_dir = "spikes"; ratname="T036";
tail="18263622"; model_type_dir = "spikes"; ratname="T035";

In [112]:
model_type_dir == "joint" ? (model_type = ["choice", "spikes"];) : model_type = model_type_dir;

In [113]:
reload_pth=pth*"/data/results/julia/"*model_type_dir*"/"*ratname*"/"*tail;

## Down to business

In [114]:
#data = load(reload_pth*"/results.jld","data");
#data["nT"] = repmat(data["nT"],10)
#data["leftbups"] = repmat(data["leftbups"],10)
#data["rightbups"] = repmat(data["rightbups"],10);

In [115]:
#create new data for sampling from model
mydt = 1e-3;

ratname == "T036" ? sessid = [157201,157357,157507,168499] : sessid = [169448,167725,166135,164900]
#ratname == "T036" ? sessid = 157201 : sessid = 166135

data, N = make_data(pth,sessid,ratname;dt=mydt);

In [116]:
ΔLR = map((x,y,z)->diffLR(x,y,z,path=true,dt=mydt),data["nT"],data["leftbups"],data["rightbups"]);

In [117]:
kind = "exp";

## Make new data

In [10]:
#create new dataset, and organize "by neuron"
N = 3;

#make sigmoid parameters
py = map(i->cat(1,log.(10*rand(1)), log.(30*rand(1)), 2*sign.(randn(2)).*rand(2)),1:N);

In [12]:
#map rate parameters
map(x->map_func_fr!(x,kind),py)

#make firing rates
λ_samp = map(neuron->map(i->my_sigmoid_2(ΔLR[i],py[neuron]),data["trial"][neuron]),1:N);

#remap
map(x->inv_map_func_fr!(x,kind),py)

#make spikes
data["spike_counts"] = map(λ->map(i->make_spikes(λ[i],dt=mydt),1:length(λ)),λ_samp);
data["trial"] = data["trial"][1:N];

ΔLR = map((x,y,z)->diffLR(x,y,z,path=true,dt=mydt),data["nT"][data["trial"][end]],
    data["leftbups"][data["trial"][end]],data["rightbups"][data["trial"][end]]);

In [13]:
#compute true loglikelihood for each neuron
pmap((p,k,trials)->ll_func(p,k,trials,ΔLR),py,data["spike_counts"],data["trial"])

3-element Array{Float64,1}:
 -2663.15
 -6515.68
 -5887.76

In [14]:
#map real parameters
map(x->map_func_fr!(x,kind),py)

3-element Array{Array{Float64,1},1}:
 [2.2948, 6.31934, 0.90298, 1.14388]  
 [7.87522, 22.3254, -1.24752, 0.29554]
 [5.01333, 25.497, -1.29608, 0.938745]

## Optimize

In [118]:
#make sigmoid parameters
py0 = cat(1,log.(1e-2),log.(1e-2), zeros(2));

In [119]:
#define optimization function
@everywhere opt_func(p0,k,trials,ΔLR) = Optim.minimizer(optimize(p0 -> -1*ll_func(p0,k,trials,ΔLR), 
        p0, method = Optim.BFGS(alphaguess = InitialStatic(alpha=1.0,scaled=true), 
        linesearch = BackTracking()), autodiff=:forward, g_tol = 1e-12, x_tol = 1e-16, 
        f_tol = 1e-16, iterations = 1000, show_trace = false, allow_f_increases = true));

In [120]:
#optimize all neurons
pystar = pmap((k,trials)->opt_func(py0,k,trials,ΔLR),data["spike_counts"],data["trial"]);

In [121]:
#compute inital loglikelihood for each neuron
pmap((k,trials)->ll_func(py0,k,trials,ΔLR),data["spike_counts"],data["trial"])

12-element Array{Float64,1}:
 -11631.8 
 -43443.5 
  -4478.75
 -34580.0 
 -10454.6 
  -5260.82
 -45013.1 
 -17739.0 
  -1941.66
 -17567.1 
 -11459.7 
 -41899.1 

In [122]:
#compute star loglikelihood for each neuron
pmap((p,k,trials)->ll_func(p,k,trials,ΔLR),pystar,data["spike_counts"],data["trial"])

12-element Array{Float64,1}:
  -6250.7 
 -18186.9 
  -2820.67
 -15418.8 
  -9079.46
  -4987.62
 -29734.0 
 -14124.3 
  -1737.28
 -15337.3 
 -10667.3 
 -30743.2 

In [123]:
#map initial parameters
map_func_fr!(py0,kind)

4-element Array{Float64,1}:
 0.01
 0.01
 0.0 
 0.0 

In [124]:
#map pystar parameters
map(x->map_func_fr!(x,kind),pystar)

12-element Array{Array{Float64,1},1}:
 [6.15955, 1.28869, -0.89926, -1.82057]         
 [23.2834, 36.9331, -0.108914, 2.94122]         
 [2.32521, 0.248572, -52.3979, -25.7417]        
 [12.4637, 4.35441e11, -0.0231066, 24.9544]     
 [7.68021, 0.600083, 41.4716, -643.639]         
 [2.4482, 1.93709, 0.0955062, -1.80033]         
 [29.8336, 6.72574, -0.542894, -9.0595]         
 [1.33742e-8, 8.20436e111, -0.00871174, 255.052]
 [1.37902, 5.30755e-13, -7.71067, -54.7821]     
 [18.5114, 7.4866e-11, 5.47224, -14.8133]       
 [11.4894, 1.92864, 0.707464, 5.40183]          
 [43.6816, 2.70904e7, -0.0546958, 16.8677]      

In [125]:
@everywhere function x0_spikes2(ΔLR,k,kind;dt::Float64=1e-3)
        
    p0 = [1e-2,1e-2,0.,0.];
    p0 = inv_map_func_fr!(p0,kind);

    x0_C(p) = sum((my_sigmoid_2(ΔLR,map_func_fr!(p,kind)) - k/dt).^2)

    od = OnceDifferentiable(x0_C,p0; autodiff=:forward)
        results = optimize(od, p0, BFGS(alphaguess = LineSearches.InitialStatic(alpha=1.0,scaled=true), 
        linesearch = BackTracking()), Optim.Options(g_tol = 1e-12, x_tol = 1e-16, 
        f_tol = 1e-16, allow_f_increases = true))   

    x0y = results.minimizer;          
    x0y = map_func_fr!(x0y,kind);
    
    return x0y
    
end

In [ ]:
pystar2 = pmap((k,trials)->x0_spikes2(vcat(ΔLR[trials]...),vcat(k...),kind;dt=mydt),
    data["spike_counts"],data["trial"]);

In [ ]:
pystar2

## Binning data and plotting

In [ ]:
nconds = 11;

conds_bins = map(x->qcut(vcat(ΔLR[x]...),nconds,labels=false,duplicates="drop",retbins=true),data["trial"]);
fr = map(j -> map(i -> (1/mydt)*mean(vcat(data["spike_counts"][j]...)[conds_bins[j][1] .== i]),0:nconds-1),1:N); #this doesn't work anymore

In [ ]:
fig, ax = subplots(N,1, figsize=(6,56))

for i = 1:N
    
    xc = conds_bins[i][2][1:end-1] + diff(conds_bins[i][2])/2;
    
    ax[i][:scatter](xc,fr[i],color="grey",label="data")
    ax[i][:plot](xc,my_sigmoid_2(xc,pystar[i]),color="red",label="new way")
    ax[i][:plot](xc,my_sigmoid_2(xc,py0),color="green",label="initial")
    #ax[i][:plot](xc,my_sigmoid_2(xc,py[i]),color="blue",label="true")
    ax[i][:plot](xc,my_sigmoid_2(xc,pystar2[i]),color="orange",label="old way")
    
    ax[i][:set_ylabel]("rate")
    ax[i][:set_xlabel](L"\Delta_{LR}(T)")
    ax[i][:set_xlim](-40,40);
    ax[i][:spines]["top"][:set_color]("none") # Remove the top axis boundary
    ax[i][:spines]["right"][:set_color]("none") # Remove the top axis boundary
    ax[i][:legend](loc="lower left")
    
end
#savefig(pth*"/figures/labmeeting/"*ratname*"_"*model_type_dir*"_tanh_psychometric.png")

## Dangling bits

In [69]:
#create new data for sampling from model

in_pth = pth*"/data/hanks_data_sessions"

data2 = Dict("leftbups" => Array{Float64}[], "rightbups" => Array{Float64}[], 
            "T" => Float64[], "hereL" => Array{Int64}[], "hereR" => Array{Int64}[],
            "nT" => Int64[], "pokedR" => Bool[], "spike_counts" => Array{Int64}[], 
            "N" => Array{Int64}[], "correct_dir" => Bool[],
            "sessid" => Int[])

t0 = 0;
N0 = 0;

for i = 1:length(sessid)
    file = matopen(in_pth*"/"*ratname*"_"*string(sessid[i])*".mat")
    rawdata = read(file,"rawdata")
    data2,t0,N0 = package_data!(data2,rawdata,["spikes"],t0,N0,dt=2e-2)
end
N = N0;

LoadError: [91mMethodError: no method matching package_data!(::Dict{String,Any}, ::Dict{String,Any}, ::Array{String,1}, ::Int64, ::Int64; dt=0.02)[0m
Closest candidates are:
  package_data!(::Any, ::Any, ::Any; dt, organize) at /Users/briandepasquale/Projects/inProgress/spike-data_latent-accum/code/julia/helpers.jl:64[39m

In [21]:
nbins2 = 20;
mSC2 = Array{Float64,2}(nbins2,N);
num_data = Array{Float64,2}(nbins2,N);
temp2 = vcat(ΔLR...);

xediffLR2 = linspace(minimum(temp2),maximum(temp2),nbins2 + 1);
xc2 = xediffLR2[1:end-1]+mean(diff(xediffLR2))/2;

for j = 1:N
    
    temp1 = vcat(data["spike_counts"][j]...);
    
    for i = 1:nbins2
        mSC2[i,j] = (1/mydt) * mean(temp1[(temp2 .>= xediffLR2[i]) .& (temp2 .< xediffLR2[i+1])]);
        num_data[i,j] = length(temp1[(temp2 .>= xediffLR2[i]) .& (temp2 .< xediffLR2[i+1])])
    end
    
end

In [27]:
betas2,mu02 = compute_priors("spikes",N)
py02 = x0_spikes(data2,N,betas2,mu02,kind,dt=2e-2)

12×4 Array{Float64,2}:
 19.9888     8.39758     -0.628975     -3.22603 
  0.964895   2.75524     -1.31598      -4.80008 
 20.2397    18.9859       0.220353      1.38118 
 20.9226     3.73841   -184.672      -211.922   
  0.940697   1.4868      -2.54405      -7.44369 
 16.062     25.932        0.0897874     0.770666
 20.6558     9.81194     -0.143027      0.481584
  1.05261    4.01621     -0.712512     -2.72006 
 14.0877     5.58256      0.476742      2.51589 
  6.10329    3.10894    -22.2338     -133.495   
 27.1032    95.8155       0.0433593     0.698334
  4.64261    0.759692    22.9164      -20.3367  

In [ ]:
#a,b, max binnec fr

In [28]:
tail="18359421"; model_type_dir = "spikes"; ratname="T036";
reload_pth=pth*"/data/results/julia/"*model_type_dir*"/"*ratname*"/"*tail;
x0 = load(reload_pth*"/results.jld","x0");

In [30]:
reshape(x0[9:end],12,4)

12×4 Array{Float64,2}:
 19.9888     8.39758    -0.628975     -3.22603 
  0.964895   2.75524    -1.31598      -4.80008 
 20.2397    18.9859      0.220353      1.38118 
 20.9226     3.73841   -90.8646     -166.707   
  0.940697   1.4868     -2.54402      -7.44361 
 16.062     25.932       0.0897876     0.770666
 20.6558     9.81194    -0.143027      0.481584
  1.05261    4.01621    -0.712512     -2.72006 
 14.0877     5.58256     0.476742      2.51589 
  6.10327    3.10895   -10.4539      -62.8157  
 27.1044    95.8108      0.0433614     0.698321
  4.64531    0.887049   23.6493       10.2385  

In [31]:
betas = Array{Float64,2}(N,4)
mu0 = Array{Float64,2}(N,4)

for j = 1:N
    betas[j,:] = [0.,0.,0.,0.];
    mu0[j,:] = [1e-2,1e-2,0.,0.];
end

betas = vec(betas);
mu0 = vec(mu0);

In [35]:
od = OnceDifferentiable(x->ll, py0; autodiff=:forward);

In [31]:
@time Optim.minimizer(optimize(x -> ll_func(x,vcat(data["spike_counts"][1]...),vcat(ΔLR[data["trial"][1]]...)), 
        py0[1,:], method = BFGS(alphaguess = LineSearches.InitialStatic(alpha=1.0,scaled=true), 
        linesearch = BackTracking()), autodiff=:forward, g_tol = 1e-12, x_tol = 1e-16, 
        f_tol = 1e-16, iterations = 1000, show_trace = false, allow_f_increases = true));

 71.085731 seconds (700.72 k allocations: 94.514 GiB, 23.59% gc time)


In [7]:
function LL_all{TT}(py::Array{TT}, data::Dict, N::Int, kind::String, ΔLR::Array{Array{Int64,1},1}; dt::Float64=2e-2)
    
    py = map(i->map_func_fr(py[i,:],kind),1:N)
    
    output = reduce(+, pmap((N,k,ΔLR) -> LL_one(py[N], k ,ΔLR, dt = dt), data["N"], data["spike_counts"], ΔLR))  
    
    return output
    
    #return -(sum(output) - sum(beta .* (vec(py) - mu0).^2))    

end

function LL_one(py,k,ΔLR;dt::Float64=2e-2)
    
    λ = map(neuron -> my_sigmoid_2(ΔLR,py[neuron]),1:length(py));      
    LL = reduce(+, map((k,λ) -> reduce(+, poiss_likelihood(k,λ,dt)), k, λ))  
    
    return LL
    
end

ll(x) = -1*LL_all(x, data, N, kind, ΔLR, dt = mydt) 

LL_all (generic function with 1 method)